In [1]:
"""
    Entry point for the main training, on 6000 images, batch size of 12, and 1000 epochs. 
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
import logging 
#from logger import Logger
import datetime
import os

from complexYOLO import ComplexYOLO
from kitti import KittiDataset
from region_loss import RegionLoss

In [2]:
date_str = str(datetime.date.today())
logging_file = 'training_log_' + date_str + '.log'
logging.basicConfig(filename = logging_file, level=logging.DEBUG, format = '%(asctime)s %(message)s')

batch_size = 12

# Remove old loggings in the tensorboard folder 
ts_dir = './logs'
for ts_file in os.listdir(ts_dir):
  ts_path = os.path.join(ts_dir, ts_file)
  os.unlink(ts_path)

# dataset
dataset=KittiDataset(root = '/home/ubuntu/KITTI', set = 'train')
data_loader = data.DataLoader(dataset, batch_size, shuffle = True, pin_memory = False)

model = ComplexYOLO()
model.cuda()

# define optimizer
optimizer = optim.SGD(model.parameters(), lr = 1e-5 ,momentum = 0.9 , weight_decay = 0.0005)

# define the number of epochs
epochs = range(200)

# Define the loss function
region_loss = RegionLoss(num_classes = 8, num_anchors = 5)
num_iters = int(len(data_loader.dataset) / batch_size)
#loss_history = np.zeros((len(epochs), num_iters, 8))
loss_history = np.zeros((1000, num_iters, 8))


In [ ]:
for epoch in epochs:
   logging.info('Running epoch = %d' % epoch)

   # Learning rate varies with epoch
   for group in optimizer.param_groups:
       if(epoch >= 4 & epoch < 80):
           group['lr'] = 1e-4
       if(epoch>=80 & epoch<160):
           group['lr'] = 1e-5
       if(epoch>=160):
           group['lr'] = 1e-6

   for batch_idx, (rgb_map, target) in enumerate(data_loader): 
          if(batch_idx == num_iters):
              break

          logging.info("Running batch_idx = %d" % batch_idx)
         
          optimizer.zero_grad()

          rgb_map = rgb_map.view(rgb_map.data.size(0),rgb_map.data.size(3),rgb_map.data.size(1),rgb_map.data.size(2))
          output = model(rgb_map.float().cuda())

          loss = region_loss(output,target, loss_history, epoch, batch_idx)
          loss.backward()

          optimizer.step()

   # Average the loss for all batches in the same epoch and log the loss	
   loss_epoch = loss_history[epoch, :, :].mean(axis = 0)
   logging.info("Epoch loss = %s" % loss_epoch)
   print('=======Eppoch {} finished, loss:  {}'.format(epoch, sum(loss_epoch))) 

   # Add tensorboard looging to monitor losses in real time
   tensorboard_info = dict(zip(['x', 'y', 'w', 'l', 'conf', 'cls', 'euler'], loss_epoch))
   tensorboard = Logger('./logs')
   tensorboard.scalar_summary(tensorboard_info, epoch)
   
   # Save model and loss every 50 epochs
   if (epoch % 2 == 0):
       logging.info("Saving model at epoch = %d" % epoch)
       torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))
       
       logging.info("Saving all losses at epoch = %d" % epoch)
       np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)

# Save model and loss at the very end
logging.info("Saving model at the last epoch = %d!" % epoch)
torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))

logging.info("Saving all losses at the last epoch = %d!" % epoch)
np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)
torch.cuda.empty_cache()


=======Epoch 0 start
-------Iteration 0 loss: 8375.56640625


/home/ubuntu/complex_yolo_3d/region_loss.py:216: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = loss_x + loss_y + loss_w + loss_l + loss_conf + loss_cls + loss_Euler + torch.tensor(loss_iou).cuda()


-------Iteration 200 loss: 1988.9866943359375
-------Iteration 400 loss: 1830.7156982421875
-------Iteration 600 loss: 1615.9527587890625
=======Eppoch 0 finished, loss:  1934.0467565016606
=======Epoch 1 start
-------Iteration 0 loss: 1945.0830078125
-------Iteration 200 loss: 1689.803955078125
-------Iteration 400 loss: 1813.08984375
-------Iteration 600 loss: 1908.880615234375
=======Eppoch 1 finished, loss:  1899.3046443774267
=======Epoch 2 start
-------Iteration 0 loss: 1564.8287353515625
-------Iteration 200 loss: 2353.2841796875
-------Iteration 400 loss: 1173.369140625
-------Iteration 600 loss: 1535.560302734375
=======Eppoch 2 finished, loss:  1881.4264037119253
=======Epoch 3 start
-------Iteration 0 loss: 1411.9024658203125
-------Iteration 200 loss: 1462.0875244140625
-------Iteration 400 loss: 1854.1861572265625
-------Iteration 600 loss: 1620.0513916015625
=======Eppoch 3 finished, loss:  1891.7634630082696
=======Epoch 4 start
-------Iteration 0 loss: 1996.665161132812

In [4]:
#date_str = str(datetime.date.today())
#logging_file = 'training_log_' + date_str + '.log'
#logging.basicConfig(filename = logging_file, level=logging.DEBUG, format = '%(asctime)s %(message)s')

batch_size = 12

# Remove old loggings in the tensorboard folder 


# dataset
dataset=KittiDataset(root = '/home/ubuntu/KITTI', set = 'train')
data_loader = data.DataLoader(dataset, batch_size, shuffle = True, pin_memory = False)

#model = ComplexYOLO()
model = torch.load("model/ComplexYOLO_epoch26")
model.cuda()

# define optimizer
optimizer = optim.SGD(model.parameters(), lr = 5e-5 ,momentum = 0.9 , weight_decay = 0.0005)

# define the number of epochs
epochs = range(27, 41)

# Define the loss function
region_loss = RegionLoss(num_classes = 8, num_anchors = 5)
num_iters = int(len(data_loader.dataset) / batch_size)
#loss_history = np.zeros((len(epochs), num_iters, 8))
loss_history = np.zeros((1000, num_iters, 8))

In [ ]:
# for epoch in epochs:
   logging.info('Running epoch = %d' % epoch)

   # Learning rate varies with epoch
   for group in optimizer.param_groups:
       if(epoch >= 4 & epoch < 80):
           group['lr'] = 1e-4
       if(epoch>=80 & epoch<160):
           group['lr'] = 1e-5
       if(epoch>=160):
           group['lr'] = 1e-6

   for batch_idx, (rgb_map, target) in enumerate(data_loader): 
          if(batch_idx == num_iters):
              break

          #logging.info("Running batch_idx = %d" % batch_idx)
         
          optimizer.zero_grad()

          rgb_map = rgb_map.view(rgb_map.data.size(0),rgb_map.data.size(3),rgb_map.data.size(1),rgb_map.data.size(2))
          output = model(rgb_map.float().cuda())

          loss = region_loss(output,target, loss_history, epoch, batch_idx)
          loss.backward()

          optimizer.step()

   # Average the loss for all batches in the same epoch and log the loss	
   loss_epoch = loss_history[epoch, :, :].mean(axis = 0)
   #logging.info("Epoch loss = %s" % loss_epoch)
   print('=======Eppoch {} finished, loss:  {}'.format(epoch, sum(loss_epoch))) 

   # Add tensorboard looging to monitor losses in real time
   #tensorboard_info = dict(zip(['x', 'y', 'w', 'l', 'conf', 'cls', 'euler'], loss_epoch))
   #tensorboard = Logger('./logs')
   #tensorboard.scalar_summary(tensorboard_info, epoch)
   
   # Save model and loss every 50 epochs
   if (epoch % 2 == 0):
       #logging.info("Saving model at epoch = %d" % epoch)
       torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))
       
       #logging.info("Saving all losses at epoch = %d" % epoch)
       np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)

# Save model and loss at the very end
#logging.info("Saving model at the last epoch = %d!" % epoch)
torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))

#logging.info("Saving all losses at the last epoch = %d!" % epoch)
np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)
torch.cuda.empty_cache()


In [5]:
#date_str = str(datetime.date.today())
#logging_file = 'training_log_' + date_str + '.log'
#logging.basicConfig(filename = logging_file, level=logging.DEBUG, format = '%(asctime)s %(message)s')

batch_size = 12

# Remove old loggings in the tensorboard folder 


# dataset
dataset=KittiDataset(root = '/home/ubuntu/KITTI', set = 'train')
data_loader = data.DataLoader(dataset, batch_size, shuffle = True, pin_memory = False)

#model = ComplexYOLO()
model = torch.load("model/ComplexYOLO_epoch40")
model.cuda()

# define optimizer
optimizer = optim.SGD(model.parameters(), lr = 5e-5 ,momentum = 0.9 , weight_decay = 0.0005)

# define the number of epochs
epochs = range(41, 51)

# Define the loss function
region_loss = RegionLoss(num_classes = 8, num_anchors = 5)
num_iters = int(len(data_loader.dataset) / batch_size)
#loss_history = np.zeros((len(epochs), num_iters, 8))
loss_history = np.zeros((1000, num_iters, 8))

In [6]:
for epoch in epochs:
   logging.info('Running epoch = %d' % epoch)

   # Learning rate varies with epoch
   for group in optimizer.param_groups:
       if(epoch >= 4 & epoch < 40):
           group['lr'] = 1e-4
       if(epoch>=40 & epoch<160):
           group['lr'] = 1e-5
       if(epoch>=160):
           group['lr'] = 1e-6

   for batch_idx, (rgb_map, target) in enumerate(data_loader): 
          if(batch_idx == num_iters):
              break

          #logging.info("Running batch_idx = %d" % batch_idx)
         
          optimizer.zero_grad()

          rgb_map = rgb_map.view(rgb_map.data.size(0),rgb_map.data.size(3),rgb_map.data.size(1),rgb_map.data.size(2))
          output = model(rgb_map.float().cuda())

          loss = region_loss(output,target, loss_history, epoch, batch_idx)
          loss.backward()

          optimizer.step()

   # Average the loss for all batches in the same epoch and log the loss	
   loss_epoch = loss_history[epoch, :, :].mean(axis = 0)
   #logging.info("Epoch loss = %s" % loss_epoch)
   print('=======Eppoch {} finished, loss:  {}'.format(epoch, sum(loss_epoch))) 

   # Add tensorboard looging to monitor losses in real time
   #tensorboard_info = dict(zip(['x', 'y', 'w', 'l', 'conf', 'cls', 'euler'], loss_epoch))
   #tensorboard = Logger('./logs')
   #tensorboard.scalar_summary(tensorboard_info, epoch)
   
   # Save model and loss every 50 epochs
   if (epoch % 2 == 0):
       #logging.info("Saving model at epoch = %d" % epoch)
       torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))
       
       #logging.info("Saving all losses at epoch = %d" % epoch)
       np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)

# Save model and loss at the very end
#logging.info("Saving model at the last epoch = %d!" % epoch)
torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))

#logging.info("Saving all losses at the last epoch = %d!" % epoch)
np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)
torch.cuda.empty_cache()


=======Epoch 41 start
-------Iteration 0 loss: 1315.03601074


KeyboardInterrupt: 

In [6]:
#date_str = str(datetime.date.today())
#logging_file = 'training_log_' + date_str + '.log'
#logging.basicConfig(filename = logging_file, level=logging.DEBUG, format = '%(asctime)s %(message)s')

batch_size = 12

# Remove old loggings in the tensorboard folder 


# dataset
dataset=KittiDataset(root = '/home/ubuntu/KITTI', set = 'train')
data_loader = data.DataLoader(dataset, batch_size, shuffle = True, pin_memory = False)

#model = ComplexYOLO()
model = torch.load("model/ComplexYOLO_epoch44")
model.cuda()

# define optimizer
#optimizer = optim.SGD(model.parameters(), lr = 1e-5 ,momentum = 0.9 , weight_decay = 0.0005)
optimizer = optim.Adam(model.parameters(), lr = 1e-5)
# define the number of epochs
epochs = range(45, 65)

# Define the loss function
region_loss = RegionLoss(num_classes = 8, num_anchors = 5)
num_iters = int(len(data_loader.dataset) / batch_size)
#loss_history = np.zeros((len(epochs), num_iters, 8))
loss_history = np.zeros((1000, num_iters, 8))


for epoch in epochs:
   logging.info('Running epoch = %d' % epoch)

   # Learning rate varies with epoch
#    for group in optimizer.param_groups:
#        if(epoch >= 4 & epoch < 40):
#            group['lr'] = 1e-4
#        if(epoch>=40 & epoch<160):
#            group['lr'] = 1e-5
#        if(epoch>=160):
#            group['lr'] = 1e-6

   for batch_idx, (rgb_map, target) in enumerate(data_loader): 
          if(batch_idx == num_iters):
              break
or epoch in epochs:
logging.info('Running epoch = %d' % epoch)

Learning rate varies with epoch
for group in optimizer.param_groups: if(epoch >= 4 & epoch < 80): group['lr'] = 1e-4 if(epoch>=80 & epoch<160): group['lr'] = 1e-5 if(epoch>=160): group['lr'] = 1e-6

for batch_idx, (rgb_map, target) in enumerate(data_loader): if(batch_idx == num_iters): break

      #logging.info("Running batch_idx = %d" % batch_idx)

      optimizer.zero_grad()

      rgb_map = rgb_map.view(rgb_map.data.size(0),rgb_map.data.size(3),rgb_map.data.size(1),rgb_map.data.size(2))
      output = model(rgb_map.float().cuda())

      loss = region_loss(output,target, loss_history, epoch, batch_idx)
      loss.backward()

      optimizer.step()
Average the loss for all batches in the same epoch and log the loss
loss_epoch = loss_history[epoch, :, :].mean(axis = 0)

#logging.info("Epoch loss = %s" % loss_epoch) print('=======Eppoch {} finished, loss: {}'.format(epoch, sum(loss_epoch)))

Add tensorboard looging to monitor losses in real time
#tensorboard_info = dict(zip(['x', 'y', 'w', 'l', 'conf', 'cls', 'euler'], loss_epoch))

#tensorboard = Logger('./logs')

#tensorboard.scalar_summary(tensorboard_info, epoch)

Save model and loss every 50 epochs
if (epoch % 2 == 0):

   #logging.info("Saving model at epoch = %d" % epoch)
   torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))

   #logging.info("Saving all losses at epoch = %d" % epoch)
   np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)
Save model and loss at the very end¶
          #logging.info("Running batch_idx = %d" % batch_idx)
         
          optimizer.zero_grad()

          rgb_map = rgb_map.view(rgb_map.data.size(0),rgb_map.data.size(3),rgb_map.data.size(1),rgb_map.data.size(2))
          output = model(rgb_map.float().cuda())

          loss = region_loss(output,target, loss_history, epoch, batch_idx)
          loss.backward()

          optimizer.step()

   # Average the loss for all batches in the same epoch and log the loss	
   loss_epoch = loss_history[epoch, :, :].mean(axis = 0)
   #logging.info("Epoch loss = %s" % loss_epoch)
   print('=======Eppoch {} finished, loss:  {}'.format(epoch, sum(loss_epoch))) 

   # Add tensorboard looging to monitor losses in real time
   #tensorboard_info = dict(zip(['x', 'y', 'w', 'l', 'conf', 'cls', 'euler'], loss_epoch))
   #tensorboard = Logger('./logs')
   #tensorboard.scalar_summary(tensorboard_info, epoch)
   
   # Save model and loss every 50 epochs
   if (epoch % 2 == 0):
       #logging.info("Saving model at epoch = %d" % epoch)
       torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))
       
       #logging.info("Saving all losses at epoch = %d" % epoch)
       np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)

# Save model and loss at the very end
#logging.info("Saving model at the last epoch = %d!" % epoch)
torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))

#logging.info("Saving all losses at the last epoch = %d!" % epoch)
np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)
torch.cuda.empty_cache()


=======Epoch 45 start
-------Iteration 0 loss: 1704.62487793
-------Iteration 200 loss: 1613.78295898
-------Iteration 400 loss: 2044.78723145
-------Iteration 600 loss: 1292.21057129
=======Eppoch 45 finished, loss:  1692.78230136
=======Epoch 46 start
-------Iteration 0 loss: 1780.33459473
-------Iteration 200 loss: 884.127624512
-------Iteration 400 loss: 1148.31274414
-------Iteration 600 loss: 1621.02978516
=======Eppoch 46 finished, loss:  1722.00764467
=======Epoch 47 start
-------Iteration 0 loss: 1105.15649414
-------Iteration 200 loss: 869.00012207
-------Iteration 400 loss: 1725.48144531
-------Iteration 600 loss: 2049.80639648
=======Eppoch 47 finished, loss:  1711.64875156
=======Epoch 48 start
-------Iteration 0 loss: 1282.03515625
-------Iteration 200 loss: 2706.73754883
-------Iteration 400 loss: 1939.37426758
-------Iteration 600 loss: 2026.06384277
=======Eppoch 48 finished, loss:  1702.07132935
=======Epoch 49 start
-------Iteration 0 loss: 2126.57128906
-------Itera

KeyboardInterrupt: 

In [2]:
#date_str = str(datetime.date.today())
#logging_file = 'training_log_' + date_str + '.log'
#logging.basicConfig(filename = logging_file, level=logging.DEBUG, format = '%(asctime)s %(message)s')

batch_size = 12

# Remove old loggings in the tensorboard folder 


# dataset
dataset=KittiDataset(root = '/home/ubuntu/KITTI', set = 'train')
data_loader = data.DataLoader(dataset, batch_size, shuffle = True, pin_memory = False)

#model = ComplexYOLO()
model = torch.load("model/ComplexYOLO_epoch54")
model.cuda()

# define optimizer
#optimizer = optim.SGD(model.parameters(), lr = 1e-5 ,momentum = 0.9 , weight_decay = 0.0005)
optimizer = optim.Adam(model.parameters(), lr = 1e-5)
# define the number of epochs
epochs = range(55, 57)

# Define the loss function
region_loss = RegionLoss(num_classes = 8, num_anchors = 5)
num_iters = int(len(data_loader.dataset) / batch_size)
#loss_history = np.zeros((len(epochs), num_iters, 8))
loss_history = np.zeros((1000, num_iters, ))


for epoch in epochs:
   logging.info('Running epoch = %d' % epoch)

   # Learning rate varies with epoch
#    for group in optimizer.param_groups:
#        if(epoch >= 4 & epoch < 40):
#            group['lr'] = 1e-4
#        if(epoch>=40 & epoch<160):
#            group['lr'] = 1e-5
#        if(epoch>=160):
#            group['lr'] = 1e-6

   for batch_idx, (rgb_map, target) in enumerate(data_loader): 
          if(batch_idx == num_iters):
              break

          #logging.info("Running batch_idx = %d" % batch_idx)
         
          optimizer.zero_grad()

          rgb_map = rgb_map.view(rgb_map.data.size(0),rgb_map.data.size(3),rgb_map.data.size(1),rgb_map.data.size(2))
          output = model(rgb_map.float().cuda())

          loss = region_loss(output,target, loss_history, epoch, batch_idx)
          loss.backward()

          optimizer.step()

   # Average the loss for all batches in the same epoch and log the loss	
   loss_epoch = loss_history[epoch, :, :].mean(axis = 0)
   #logging.info("Epoch loss = %s" % loss_epoch)
   print('=======Eppoch {} finished, loss:  {}'.format(epoch, sum(loss_epoch))) 

   # Add tensorboard looging to monitor losses in real time
   #tensorboard_info = dict(zip(['x', 'y', 'w', 'l', 'conf', 'cls', 'euler'], loss_epoch))
   #tensorboard = Logger('./logs')
   #tensorboard.scalar_summary(tensorboard_info, epoch)
   
   # Save model and loss every 50 epochs
   if (epoch % 2 == 0):
       #logging.info("Saving model at epoch = %d" % epoch)
       torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))
       
       #logging.info("Saving all losses at epoch = %d" % epoch)
       np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)

# Save model and loss at the very end
#logging.info("Saving model at the last epoch = %d!" % epoch)
torch.save(model, "model/ComplexYOLO_epoch" + str(epoch))

#logging.info("Saving all losses at the last epoch = %d!" % epoch)
np.save("loss/complexYOLO_epoch" + str(epoch), loss_history)
torch.cuda.empty_cache()


region_loss.py:216: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = loss_x + loss_y + loss_w + loss_l + loss_conf + loss_cls + loss_Euler + torch.tensor(loss_iou).cuda()


=======Epoch 55 start
-------Iteration 0 loss: 1502.82568359
-------Iteration 200 loss: 1530.10302734
-------Iteration 400 loss: 1686.76025391
-------Iteration 600 loss: 1676.02392578
=======Eppoch 55 finished, loss:  1716.91826859
=======Epoch 56 start
-------Iteration 0 loss: 1173.19689941
-------Iteration 200 loss: 1984.96630859
-------Iteration 400 loss: 1787.05480957
-------Iteration 600 loss: 1631.59265137
=======Eppoch 56 finished, loss:  1747.74293974
